In [1]:
import sys
sys.path.insert(0, "scripts")

start = datetime.date(2021, 1, 1)
end = datetime.date(2021, 2,1)
lb = load_bars(start, end, 1)
lb.tail()

,,open,high,low,close,volume,amount,price
date,asset,,,,,,,
2021-01-26,000001.XSHE,22.299999,23.320000,22.299999,22.370001,112672055.0,2.558576e+09,22.309999
2021-01-27,000001.XSHE,22.309999,23.469999,22.309999,23.080000,129415272.0,2.976801e+09,22.780001
2021-01-28,000001.XSHE,22.780001,23.180000,22.450001,22.809999,85747696.0,1.948881e+09,22.809999
2021-01-29,000001.XSHE,22.809999,23.540001,22.709999,23.090000,124025841.0,2.864101e+09,23.000000
2021-02-01,000001.XSHE,23.000000,24.990000,22.700001,24.549999,147523930.0,3.529557e+09,NaN


In [2]:
lbt = load_bars_tushare(start, end, ["000001.XSHE"])
lbt.tail()

,,open,high,low,close,volume,amount,price
date,asset,,,,,,,
1970-01-01 05:36:50.126,000001.XSHE,22.30,23.32,22.30,22.37,1126720.55,2558575.511,22.31
1970-01-01 05:36:50.127,000001.XSHE,22.31,23.47,22.31,23.08,1294152.72,2976800.955,22.78
1970-01-01 05:36:50.128,000001.XSHE,22.78,23.18,22.45,22.81,857476.96,1948881.146,22.81
1970-01-01 05:36:50.129,000001.XSHE,22.81,23.54,22.71,23.09,1240258.41,2864101.419,23.00
1970-01-01 05:36:50.201,000001.XSHE,23.00,24.99,22.70,24.55,1475239.30,3529556.986,NaN


In [5]:
lbt.compare(lb)

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects